#### Inferential Statistics Concepts

in this chapter, we'll treat a model parameter, such as slope, not as a single value but instead as a distribution of values whose mean will give the best value 
random sampling will be used to estimate the parameted distributions, the distributions will be used to make probabilistic statements about both model parameters and model predictinos 

a statistic summarizes a distribution, for example the mean temperature or the median height

population statistics and sample statistics are not usually the same, for example if sample temperatures were all from summer days then the sample mean would be higher than the population mean computed from a full year

to ensure the sample is representative of the population, both having about about the same center and spread, you can randomly draw points from the population using np.choice()

to verify that the sample is representative of the population, we plot a histogram of both 
doing this with raw counts is usually not useful because the sample count is often much smaller than the population 
if you normalize both population and sample, dividing sample bins by 31 (days) and population bins by 3650, each distribution sums to 1, then the difference in normalized distributions becomes clear 

the shape of a sample distribution is often used to make inferences about the population distribution
if you divide each bin count by the total number of days in the data set the sum across all bins is 1
so the sum of the half to the right of the 100 degrees would be about 0.5
from this normalized distribution, rather than just stating that there's some uncertainty, we can infer more precisely that there is a 50% probability that any day in August will exceed 100 degrees 

if you use np.random.choice() again to take a second sample then you'd see that the two samples differ, you could resample 100 times
think of any measured data set as a sample randomly drawn from a larger population 

In [ ]:
# population statistics versus sample statistics 
print(len(month_of_temps), month_of_temps.mean(), month_of_temps.std())
print(len(decade_of_temps), decade_of_temps.mean(), decade_of_temps.std())

In [ ]:
# draw a random sample from a population
month_of_temps = np.random.choice(decade_of_temps, size=31)

In [ ]:
# resampling as iteration, take 20 different randomly chosen samples 
num_samples = 20
for ns in range(num_samples):
    sample = np.random.choice(population, num_pts)
    distribution_of_means[ns] = sample.mean()
    
# sample distribution statistics, characterize the shape of the distribution by taking the mean and st. dev
mean_of_means = np.mean(distribution_of_means)
stdev_of_means = np.std(distribution_of_means)

In [ ]:
# example exercise, sample statistics versus population 
# Compute the population statistics (the mean and st dev)
print("Population mean {:.1f}, stdev {:.2f}".format( population.mean(), population.std() ))

# Set random seed for reproducibility
np.random.seed(42)

# Construct a sample by randomly sampling of 31 points from the population
sample = np.random.choice(population, size=31)

# Compare sample statistics to the population statistics
print("    Sample mean {:.1f}, stdev {:.2f}".format( sample.mean(), sample.std() ))
# you'll see that the sample stats are similar to the pop stats
# if you were to compute len() of each array, it's very different, the the means are not that much different as you might expect 

In [ ]:
# example exercise, variation in sample statistics
# Initialize two arrays of zeros to be used as containers
means = np.zeros(num_samples)
stdevs = np.zeros(num_samples)

# For each iteration, compute and store the sample mean and sample stdev
for ns in range(num_samples):
    sample = np.random.choice(population, num_pts)
    means[ns] = sample.mean()
    stdevs[ns] = sample.std()

# Compute and print the mean() and std() for the sample statistic distributions
print("Means:  center={:>6.2f}, spread={:>6.2f}".format(means.mean(), means.std()))
print("Stdevs: center={:>6.2f}, spread={:>6.2f}".format(stdevs.mean(), stdevs.std()))

# if only a single sample of 100 was taken then there could only be a single mean and the st dev of that single value would be 0
# each sample will be different because of the random draws
# the mean of the means is the estimate for the population mean
# the stdev of the means is the measure of the uncurtainty in our estimate of the population mean
# this is the same concept as the standard error of the slope seen in linear regression

In [ ]:
# example exercise, visualize the variation of a statistic
# Generate sample distribution and associated statistics
means, stdevs = get_sample_statistics(population, num_samples=100, num_pts=1000)

# Define the binning for the histograms
mean_bins = np.linspace(97.5, 102.5, 51)
std_bins = np.linspace(7.5, 12.5, 51)

# Plot the distribution of means, and the distribution of stdevs
fig = plot_hist(data=means, bins=mean_bins, data_name="Means", color='green')
fig = plot_hist(data=stdevs, bins=std_bins, data_name="Stdevs", color='red')
# you'll see that sample statistics have a distribution of values, not just a single value 

#### Model Estimation and Likelihood

you want to model the population but you only have a sample of the data
in this lesson use estimation to build models of population distributions from sampple statistics 

an example: if you measured the distance traveled by a satellite each hour for a week (the gray bars), the build a model of the distribution of distances traveled in each hour for an entire year (the red curve), to build that model, the assumptions are
1. the population model is shaped like a Gaussian
2. the sample statistics are good estimates for the population model parameters 

(do the estimatio code below)
why do we do this? what is the liklihood that this specific model best predicts our given data? 

Likelihood vs Probability

conditional probability
: stated as a question of what is the probability that A occurs, given the condition that B has already occurred? P(outcome A|given B)

probability
: if the model is given then we ask what is the probability that it outputs any particular data point? P(data|model)

likelihood
: if the data is given then we ask what is the likelihood that a candidate model could output the particular data we have? L(model|data)


if you had two candidate models then you'd want to choose the one that has the greatest likelihood to output the given data 

how is liklihood computed?
start with a gaussian model (red bell curve line thingy) with specific values for its parameters (mu and sigma)
with the condition that this model is given, we ask what is the probability that this model would output one particula data point? for a single distance (value on the x-axis), the probability that the model outputs the data is shown as the horizontal green line (y-value where the value on the x-axis and part of the curve meet)
repeat that process for every distance in the sample and take the product 
if the model peak is centered over most of the sample data then the product of probabilities will be large, but if the sample distances are far away from the model peak (out under the wings of the model) then the likelihood that the model could produce the data set is small 

In [ ]:
# estimation
# define gaussian model function, mu and sigma are the center and spread
def gaussian_model(x, mu, sigma):
    coeff_part = 1 / (np,sqrt(2 * np.pi * sigma**2))
    exp_part = np.exp(-(x - mu)**2 / (2 * sigma**2))
    return coeff_part * exp_part

# compute sample statistics, the sample mean and the sample standard deviation
mean = np.mean(sample)
stdev = np.std(sample)

# model the population using sample statistics, use the sample mean and stdev as estimates of the population parameters (mu and sigma)
population_model = gaussian_samodel(sample, mu=mean, sigma=stdev)

In [ ]:
# likelihood from probabilities
# in code, we start by using the sample statistics as guesses for the population model parameters (mu and sigma)

# guess parameters
mu_guess = np.mean(sample_distances)
sigma_guess = np.std(sample_distances)

# for each sample point compute a probability by passing the distance and guesses for mu and sigma into the model
probabilities = np.zeros(len(sample_distances))
for n, distance in enumerate(sample_distances):
    probabilities[n] = gaussian_model(distance, mu=mu_guess, sigma=sigma_guess)
    
# take the product of all those probabilities to compute likelihood
likelihood = np.product(probs)
# it's useful to take the log of the likelihood because it has better numerical properties
loglikelihood = np.sum(np.log(probs))

In [ ]:
# maximum likelihood estimation
# now repeat the process for an array of models
# try 101 guesses, centered on the sample mean
low_guess = sample_mean - 2*sample_stdev
high_guess = sample_mean + 2*sample_stdev
mu_guessus = np.linspace(low_guess, high_guess, 101)

# for each iteration we compute loglikelihood which results in an array of 101 loglikelihoods
loglikelihood = np.zeros(len(mu_guesses))
for n, mu_guesses in enumerate(mu_guesses):
    loglikelihoods[n] = compute_loglikelihood(sample_distances, mu=mu_guesses, sigma=sample_stdev)
    
# find the best guess (the one guess that gives the maximum loglikelihood)
max_loglikelihood = np.max(loglikelihoods)
best_mu = mu_guesses[loglikelihoods == max_loglikelihood]

# plot the 101 loglikelihood values, one for each guess of mu, and the best estimator for the population
# mu is the guess that gives the maximum loglikelihood 
# when the model is gaussian, this mean matches the answer we'd get from least-squares 

#### Model Uncertainty and Sample Distributions

so far, to estimate a model parameter we've assumed a shape of the parameter distribution: least squares assumes a gaussian shape, maximum likelihood estimations requires us to choose a shape so we went with gaussian
but what about situations where the distribution shape is unknown? usually you'll only have a single sample and no idea about the shape of the entire population 

we could use the sample as the model of the population 
computing the mean of the single sample will give you a guess but you still have no knowledge of the uncertainty of that guess
so what we really want is a prediction for what happens when we take the next sample of a population 
we could use **bootstrap resampling** which lets you resample the sample many times to simulate taking samples from the population
each time you resample, you compute a statistic: the mean
sampling the sample then results in a distribution of sample statistic values 
we can use that to predict that a future sample mean will be 100 degrees with a probability of about 95% the mean will occur between 92 and 107 since 95% is the fraction of the area (or counts) between those values  

the *replace* used in the code below is like sampling by putting the marble back in the bag between each sigle draw so that each draw will be from the entire pool of original marbles, not putting the marble back in before making your next draw is sampling without replacement 
sampling without replacement has two issues: you never have the change to draw a repeated marble (the same marble twice in a row), and the sampling method is changing the model after every draw
to estimate the shape of a single population model using resampling thon you have to hold the model population constant (meaning you have to put the marble back in the bag between every draw)

In [ ]:
# bootstrap
# assume we have only a sample of daily high temps and 
# we want to model the August daily highs for a decade, aka predict the next sample draw from that population

# use sample as model for population, assign the sample as the model for the unmeasured population
population_model = august_daily_highs_for_2017

# simulate repeated data acquisitions by resampling the "model", resample the sample and comput the means
for nr in range(num_resamples):
    bootstrap_sample = np.random.choice(population_model, size=resample_size, replace=True)
    bootstrap_means[nr] = np.mean(bootstrap_sample)
    # the result will be many mean values, a bootstrap sample distribution

# compute the mean of the bootstrap resample distribution, compute the mean of the means
estimate_temperature = np.mean(bootstrap_means)
# the result is our best estimate of the daily high temperature in any August

# compute standard deviation of the bootstrap resample distribution
estimate_uncertainty = np.std(bootstrap_means)
# the standard deviatio of the means is the standard error or uncertainty 

# Plot the bootstrap resample distribution of means
fig = plot_data_hist(bootstrap_means)

#### Model Errors and Randomness

we've seen linear model parameters as distributions, spread about some central peak
now we'll relate the parameter distributions to the standard error of linear model parameters
we'll also check whether our parameter estimates are effected by randomness 

there are 3 types of errors common to sampling and measurement
- measurement error: mistakes make when collecting or recording the data (a broken censor, wrote down measured values incorrectly)
- sampling bias: taking draws from one small portion of the population not representative of the rest (temps only from August when it's super hot to represent the whole year
- random choice: variation, how do we know that the mean slope from the model fit is not due to just random fluctuations or noise? 

null hypothesis 
that question can be restated as: is our effect due to a relationship or due to random chance?
does the ordering or grouping of the data cause an effect larger than what could be produced by randomly shuffled data?
We interpret the "zero effect size" to mean that if we shuffled samples between short and long times, so that two new samples each have a mix of short and long duration trips, and then compute the test statistic, on average it will be zero.

this lesson will focus on answering this question with the null hypothesis 

p-value is the chance that the effect (or speed) we estimated was the result of random variation in the sample, it's a fraction

In [ ]:
# test statistics
# group into early and late times (less than or greater than 5 hours)
group_short = sample_distances[times < 5]
group_long = sample_distances[times > 5]

# resample distributions
resample_short = np.random.choice(group_short, size=500, replace=True)
resample_long = np.random.choice(group_long, size=500, replace=True)

# test statistic
# as a test statistics we compute the difference in distance between two randomly chose points, one for each group, and repeat 500 times
test_statistic = resample_long - resample_short

# effect size as mean of test statistic distribution
effect_size = np.mean(test_statistic)
# we take the mean of these differences to be our effect size for how increasing time effects a change in distance 

# now reshuffle the data, removing any sense of time grouping, in this example you'll see that the two distributions of shuffled
# group distances are entirely overlapped, you can see that the same test will likely average to 0

In [ ]:
# null hypothesis
# shuffle and split
shuffle_bucket = np.concatenate((group_short, group_long))
np.random.shuffle(shuffle_bucket)

# split in the middle, slice the shuffled data into 2
slice_index = len(shuffled_backet)//2
shuffled_half1 = shuffle_bucket[0:slice_index]
shuffled_half2 = shuffle_bucket[slice_index+1:]

# resample shuffled populations, resample the arbitrary half of the shuffled data 
shuffled_sample1 = np.random.choice(shuffled_half1, size=500, replace=True)
shuffled_sample2 = np.random.choice(shuffled_half2, size = 500, replace=True)

# recompute the test statistic and effect size
shuffled_test_statistic = shuffled_sample_2 - shuffled_sample1
effect_size = np.mean(shuffled_test_statistic)

# plot what, if any, change came from shuffling, plot the two test statistic distributions

In [ ]:
# visualizing test statistics
# From the unshuffled groups, compute the test statistic distribution
resample_short = np.random.choice(group_duration_short, size=500, replace=True)
resample_long = np.random.choice(group_duration_long, size=500, replace=True)
test_statistic_unshuffled = resample_long - resample_short

# Shuffle two populations, cut in half, and recompute the test statistic
shuffled_half1, shuffled_half2 = shuffle_and_split(group_duration_short, group_duration_long)
resample_half1 = np.random.choice(shuffled_half1, size=500, replace=True)
resample_half2 = np.random.choice(shuffled_half2, size=500, replace=True)
test_statistic_shuffled = resample_half2 - resample_half1

# Plot both the unshuffled and shuffled results and compare
fig = plot_test_statistic(test_statistic_unshuffled, label='Unshuffled')
fig = plot_test_statistic(test_statistic_shuffled, label='Shuffled')

In [ ]:
# visualizing the p-value
# the goal is to visualize this as the fraction of points in the shuffled test statistic distribution that fall to the right of
# the mean of the test statistic (effect size) computed from the unshuffled samples

# Compute the test stat distribution and effect size for two population groups
test_statistic_unshuffled = compute_test_statistic(group_duration_short, group_duration_long)
effect_size = np.mean(test_statistic_unshuffled)

# Randomize the two populations, and recompute the test stat distribution
shuffled_half1, shuffled_half2 = shuffle_and_split(group_duration_short, group_duration_long)
test_statistic_shuffled = compute_test_statistic(shuffled_half1, shuffled_half2)

# Compute the p-value as the proportion of shuffled test stat values >= the effect size
condition = test_statistic_shuffled >= effect_size
p_value = len(test_statistic_shuffled[condition]) / len(test_statistic_shuffled)

# Print p-value and overplot the shuffled and unshuffled test statistic distributions
print("The p-value is = {}".format(p_value))
fig = plot_test_stats_and_pvalue(test_statistic_unshuffled, test_statistic_shuffled)